In [1]:
from faiss import IndexFlatIP
import numpy as np


In [2]:
clip = dict(img_embeds=np.load("../coco_train2017_image_embeds.npy"),
            text_embeds=np.load("../coco_train2017_text_embeds.npy"))

cloob = dict(img_embeds=np.load("../cloob-vit-b-16/cloob_laion_400m_vit_b_16_32_epochs_coco_train2017_image_embeds.npy"),
             text_embeds=np.load("../cloob-vit-b-16/cloob_laion_400m_vit_b_16_32_epochs_coco_train2017_text_embeds.npy"))



In [3]:
def retrieval(img_embeds, text_embeds, subset_n=50000, top_k=5):
    img_embeds = img_embeds[:subset_n]
    text_embeds = text_embeds[:subset_n]
    
    n, channels = img_embeds.shape
    
    img_index = IndexFlatIP(channels)
    img_index.add(img_embeds)
    
    text_index = IndexFlatIP(channels)
    text_index.add(text_embeds)
    
    ground_truth = np.arange(n)
    
    img_dists, img_idxs = img_index.search(text_embeds, top_k)
    text_dists, text_idxs = text_index.search(img_embeds, top_k)
    
    print("text->img retrieval @1", (img_idxs[:, 0] == ground_truth).sum() / n)
    print("img->text retrieval @1", (text_idxs[:, 0] == ground_truth).sum() / n)

    print(f"text->img retrieval @{top_k}", (img_idxs == ground_truth[:, None]).any(1).sum() / n)
    print(f"img->text retrieval @{top_k}", (text_idxs == ground_truth[:, None]).any(1).sum() / n)
    

retrieval(**clip)

text->img retrieval @1 0.0698
img->text retrieval @1 0.09028
text->img retrieval @5 0.17204
img->text retrieval @5 0.2051


In [4]:
retrieval(**cloob)

text->img retrieval @1 0.12238
img->text retrieval @1 0.1298
text->img retrieval @5 0.2573
img->text retrieval @5 0.27536
